In [1]:
# import argparse
from utils import *
from torch.utils.data.dataset import *
from torch.utils.data.sampler import *  #用于采样
from torch.nn.utils.rnn import *
import bisect
# from model import *
import torch
import os
import torch.nn as nn
import torch.optim as optim
import numpy as np
import time


In [2]:
def myget_triplets(tags,outsent):
    temp = {}
    triplets = []
    for idx, tag in enumerate(tags):
        if tag == tag_set["O"]:
            continue
        entity=[]
        pos, relation_label, role = tag_set[tag].split("-")
        if pos == "B" or pos == "S":
            entity=[]
            entity.append(idx)
        if pos=="I" or pos=="E":
            entity.append(idx)
        if pos=="S" or pos=="E":
            if relation_label not in temp:
                temp[relation_label] = [[], []]
            temp[relation_label][int(role) - 1].append(entity)

    for relation_label in temp:
        role1, role2 = temp[relation_label]  #role1是第一个候选实体列表  role2是第二个候选实体列表
        if role1 and role2:
            len1, len2 = len(role1), len(role2)
            if len1 > len2:
                for e2 in role2:
                    idx = np.argmin([abs(e2[0] - e1[0]) for e1 in role1])     #实体对以就近原则进行匹配
                    e1 = role1[idx]
                    entity1=''.join([outsent[i] for i in e1])
                    entity2=''.join([outsent[i] for i in e2])
                    triplets.append((entity1, relation_label, entity2))
                    del role1[idx]
            else:
                for e1 in role1:
                    idx = np.argmin([abs(e2[0] - e1[0]) for e2 in role2])     #实体对以就近原则进行匹配
                    e2 = role2[idx]
                    entity1=''.join([outsent[i] for i in e1])
                    entity2=''.join([outsent[i] for i in e2])
                    triplets.append((entity1, relation_label, entity2))
                    del role2[idx]
    return triplets

In [3]:
### 处理tagset
tag_set = Index()#定义对象
tag_set.load("my_tag2id.txt")#读文件  标记-id

In [9]:
### 处理relation
relation_labels = Index()#定义对象
relation_labels.load('my_relation_labels.txt')#读文件   关系标签

In [19]:
output = torch.Tensor([0,0,0,0,10,0,0,0,9])
output = torch.argmax(output, dim=-1)  #返回指定维度的最大值的序号

output

tensor(4)

In [20]:
with open('mytrain_seg.txt','r',encoding='utf-8') as f:
    lines = f.readlines()
f.close()

In [23]:
import pickle

with open('my_train.pk', 'rb') as f1:
    train_data = pickle.load(f1)
f1.close()

In [24]:
len(lines) == len(train_data)

True

In [41]:
lines[1].split(' ')[-4]

'出'

In [44]:
tags = train_data[0][2]

In [50]:
outsent = lines[0].strip().split(' ')

In [46]:

temp = {}
triplets = []
for idx, tag in enumerate(tags):
    if tag == tag_set["O"]:
        continue
    entity=[]
    pos, relation_label, role = tag_set[tag].split("-")
    if pos == "B" or pos == "S":
        entity=[]
        entity.append(idx)
    if pos=="I" or pos=="E":
        entity.append(idx)
    if pos=="S" or pos=="E":
        if relation_label not in temp:
            temp[relation_label] = [[], []]
        temp[relation_label][int(role) - 1].append(entity)
        
print(entity)

print('*******************')

print(temp)

# for relation_label in temp:
#     role1, role2 = temp[relation_label]  #role1是第一个候选实体列表  role2是第二个候选实体列表
#     if role1 and role2:
#         len1, len2 = len(role1), len(role2)
#         if len1 > len2:
#             for e2 in role2:
#                 idx = np.argmin([abs(e2[0] - e1[0]) for e1 in role1])     #实体对以就近原则进行匹配
#                 e1 = role1[idx]
#                 entity1=''.join([outsent[i] for i in e1])
#                 entity2=''.join([outsent[i] for i in e2])
#                 triplets.append((entity1, relation_label, entity2))
#                 del role1[idx]
#         else:
#             for e1 in role1:
#                 idx = np.argmin([abs(e2[0] - e1[0]) for e2 in role2])     #实体对以就近原则进行匹配
#                 e2 = role2[idx]
#                 entity1=''.join([outsent[i] for i in e1])
#                 entity2=''.join([outsent[i] for i in e2])
#                 triplets.append((entity1, relation_label, entity2))
#                 del role2[idx]
# # return triplets

[27]
*******************
{'provide_shelter_for': [[[12]], [[27]]]}


In [53]:
outsent[12]

'简'

In [52]:
lines[0]

'公诉 机关 指控 ， 2015年 11月份 的 一 天 晚上 ， 被告人 简 某某 容留 陈某 、 吴某 、 杨 某某 、 张某 、 伍 某某 在 简 某某 家 一 楼 卧室 ， 以 烫 吸 的 方式 吸食 甲基苯丙胺 （ 冰毒 ） 。\n'